In [ ]:
# Unified search helper: optional upload flow + smart search with thumbnails
import os, time
import cv2
from PIL import Image
from IPython.display import display

# Assume engine is already initialized in prior cells
try:
    engine
except NameError:
    from video_search_engine import VideoSearchEngine
    engine = VideoSearchEngine()

# Helper to process a video if provided; else skip

def process_video_if_any(video_path: str = None, video_name: str = None):
    if video_path and os.path.exists(video_path):
        name = video_name or os.path.splitext(os.path.basename(video_path))[0]
        print(f"\n🎬 Processing video: {name}")
        stats = engine.process_video(
            video_path=video_path,
            video_name=name,
            save_frames=False,
            upload_to_pinecone=True,
            use_object_detection=False
        )
        print("✅ Processing complete.")
        return name, stats
    else:
        print("ℹ️ No video uploaded. Searching existing Pinecone index.")
        return None, None


def search_and_display(query: str, video_filter: str = None, top_k: int = 5):
    print(f"\n🔎 Query: {query}")
    results = engine.smart_search(
        query=query,
        top_k=top_k,
        video_filter=video_filter
    )
    if not results:
        print("No results found.")
        return

    for i, r in enumerate(results, 1):
        ts = r.get('timestamp', 0.0)
        print(f"{i}. ⏱ {r.get('time_formatted','')}  📊 {r.get('similarity_score',0):.0%}")
        print(f"   📝 {r.get('caption','')}")
        print(f"   🎥 {r.get('video_name','')}")
        thumb_path = r.get('thumbnail_path')
        if thumb_path and os.path.exists(thumb_path):
            try:
                img = Image.open(thumb_path)
                display(img)
            except Exception:
                pass
        elif r.get('video_name') in getattr(engine, 'video_paths', {}):
            # Fallback: extract and show directly
            vpath = engine.video_paths[r['video_name']]
            cap = cv2.VideoCapture(vpath)
            if cap.isOpened():
                fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
                frame_num = int(max(0, ts) * fps)
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                ok, frame = cap.read()
                cap.release()
                if ok and frame is not None:
                    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    display(Image.fromarray(rgb))
    return results



### Pinecone setup: dual indexes (text + image)
Ensure two Pinecone indexes exist and match your config:
- Text index: dimension must match your text embedding model (default 1024)
- Image index: dimension must match CLIP image/text space (default 512)

Run the cell below to verify/create the indexes automatically using values from `video_search_config.Config`. You need `PINECONE_API_KEY` set.


In [ ]:
# Verify or create Pinecone text/image indexes per config
import os, time
from video_search_config import Config
from pinecone import Pinecone, ServerlessSpec

assert Config.PINECONE_API_KEY, "PINECONE_API_KEY is required"

pc = Pinecone(api_key=Config.PINECONE_API_KEY)
existing = {idx.name for idx in pc.list_indexes()}

needed = [
    (Config.PINECONE_TEXT_INDEX_NAME, Config.PINECONE_TEXT_DIMENSION),
    (Config.PINECONE_IMAGE_INDEX_NAME, Config.PINECONE_IMAGE_DIMENSION),
]

for name, dim in needed:
    if name in existing:
        print(f"✅ Index exists: {name}")
        # sanity check dimension (best-effort)
        try:
            idx = pc.Index(name)
            stats = idx.describe_index_stats()
            actual_dim = stats.get('dimension')
            if actual_dim and int(actual_dim) != int(dim):
                print(f"⚠️ Dimension mismatch for {name}: index={actual_dim} config={dim}")
            else:
                print(f"   Dimension OK: {dim}")
        except Exception as e:
            print(f"   (warn) Could not verify dimension for {name}: {e}")
    else:
        print(f"📦 Creating index: {name} (dim={dim})")
        pc.create_index(
            name=name,
            dimension=int(dim),
            metric=Config.PINECONE_METRIC,
            spec=ServerlessSpec(cloud=Config.PINECONE_CLOUD, region=Config.PINECONE_REGION)
        )
        # brief wait
        time.sleep(5)
        print(f"   ✅ Created: {name}")

print("Done.")


# Video Frame Search System with InstructBLIP & Pinecone

This notebook sets up a complete video semantic search engine that:
- Extracts frames from videos
- Generates object-focused captions using InstructBLIP (better than BLIP for object attributes)
- Stores embeddings in Pinecone
- Enables natural language search with temporal bootstrapping

**Features:**
- 🎯 Object-focused captioning (detects colors, attributes like "black backpack", "red shirt")
- 🚀 Temporal bootstrapping (finds related objects at similar timestamps)
- 📊 Adaptive search windows (adjusts based on motion)
- ⚡ Confidence-aware boosting

---


 ## Step 1: Setup - Clone Repository & Install Dependencies



In [ ]:
# Clone the repository
!git clone https://github.com/pranavacchu/capstone-BLIP.git
%cd capstone-BLIP

# Install dependencies
print("📦 Installing dependencies... This will take 3-5 minutes")
%pip install -q opencv-python-headless pillow numpy pandas tqdm python-dotenv
%pip install -q torch torchvision transformers sentence-transformers
%pip install -q pinecone FlagEmbedding
print("\n✅ Installation complete!")

# Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"\n🚀 GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️ No GPU detected. Using CPU (slower but works)")


Cloning into 'capstone-BLIP'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 69 (delta 32), reused 55 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 100.55 KiB | 5.03 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/capstone-BLIP/capstone-BLIP/capstone-BLIP/capstone-BLIP/capstone-BLIP/capstone-BLIP/capstone-BLIP/capstone-BLIP/capstone-BLIP
📦 Installing dependencies... This will take 3-5 minutes

✅ Installation complete!

🚀 GPU detected: Tesla T4
   Memory: 15.8 GB


In [ ]:
# Verify imports work correctly
try:
    from video_search_engine import VideoSearchEngine
    print("✅ All modules loaded successfully!")
    print("📝 Using InstructBLIP model for object-focused captioning")
except Exception as e:
    print(f"⚠️ Import error: {e}")
    print("   Please check that all files are in the correct directory")


🔧 Applying hotfixes...
   - Adding deduplicate_embeddings method...
   ✓ deduplicate_embeddings already exists
   - Fixing Grounding DINO dtype mismatch...
   ⚠ Could not find code to replace in object_detector.py

✅ Hotfixes applied successfully!
   You can now proceed with video processing


In [ ]:
# Cell removed - all imports verified in cell 3

✅ Reloaded


In [ ]:
# Cell removed - all dependencies installed in cell 2

Installing Grounding DINO and dependencies...
This may take 2-3 minutes...

Grounding DINO dependencies installed!
Models will be downloaded automatically from Hugging Face on first use


## Step 2: Configure Pinecone API Key

The system will load credentials from a `.env` file. If the file doesn't exist, you'll be prompted to create it.

**Required settings:**
- **API Key**: Your Pinecone API key
- **Index Host**: Your index URL (from Pinecone dashboard)
- **Environment**: Usually `us-east-1` or your Pinecone region

In [ ]:
import os
from dotenv import load_dotenv

# Check if .env file exists
env_exists = os.path.exists('.env')

if not env_exists:
    print("📝 No .env file found. Creating one...")
    print("\nPlease enter your Pinecone credentials:")
    
    # Get credentials from user
    api_key = input("Enter your Pinecone API Key: ").strip()
    host = input("Enter your Pinecone Host URL (e.g., https://xxx.svc.xxx.pinecone.io): ").strip()
    environment = input("Enter your Pinecone Environment (e.g., us-east-1, default=us-east-1): ").strip() or "us-east-1"
    
    # Write to .env file
    with open('.env', 'w') as f:
        f.write(f"PINECONE_API_KEY={api_key}\n")
        f.write(f"PINECONE_HOST={host}\n")
        f.write(f"PINECONE_ENVIRONMENT={environment}\n")
    
    print("\n✅ .env file created!")
else:
    print("✅ .env file found. Loading credentials...")

# Load environment variables from .env file
load_dotenv()

# Verify credentials are loaded
api_key = os.getenv('PINECONE_API_KEY')
host = os.getenv('PINECONE_HOST')

if api_key and host:
    print("✅ Credentials loaded successfully!")
    print(f"   API Key: {api_key[:20]}...{api_key[-10:] if len(api_key) > 30 else ''}")
    print(f"   Host: {host}")
else:
    print("⚠️ Warning: Some credentials are missing from .env file")
    print("   Please check your .env file and ensure it contains:")
    print("   - PINECONE_API_KEY")
    print("   - PINECONE_HOST")
    print("   - PINECONE_ENVIRONMENT")

✅ Configuration saved!


##  Step 3: Test Connection to Pinecone



In [ ]:
from video_search_engine import VideoSearchEngine

print("🔌 Connecting to Pinecone...")
engine = VideoSearchEngine()

# Get database stats
stats = engine.get_index_stats()

print("\n✅ Successfully connected to Pinecone!")
print(f"\n📊 Database Statistics:")
print(f"   Index: capstone")
print(f"   Total vectors: {stats.get('total_vectors', 0):,}")
print(f"   Dimension: {stats.get('dimension', 1024)}")
print(f"   Capacity: Serverless")

🔌 Connecting to Pinecone...

✅ Successfully connected to Pinecone!

📊 Database Statistics:
   Index: capstone
   Total vectors: 35
   Dimension: 1024
   Capacity: Serverless


## Step 4: Upload a Video File



In [82]:
from google.colab import files
import os
import subprocess
from urllib.parse import urlparse, parse_qs

print("📤 Choose how to get your video:\n")
print("1. Upload from computer (recommended for small files < 100MB)")
print("2. Download from URL (direct video file)")
print("3. Download from YouTube URL\n")

choice = input("Enter choice (1/2/3): ").strip()
video_path = None

if choice == "1":
    print("\n📁 Please select your video file...")
    uploaded = files.upload()
    if uploaded:
        video_path = list(uploaded.keys())[0]
        print(f"✅ Uploaded: {video_path}")
    else:
        print("❌ No file uploaded")

elif choice == "2":
    video_url = input("\nEnter video URL (direct link to .mp4, .avi, etc.): ").strip()

    if not video_url:
        print("❌ No URL provided")
    else:
        # Extract filename from URL or use default
        parsed_url = urlparse(video_url)
        url_filename = os.path.basename(parsed_url.path)

        # Use URL filename if it has an extension, otherwise use default
        if url_filename and '.' in url_filename:
            video_filename = url_filename
        else:
            video_filename = "downloaded_video.mp4"

        print(f"⬇️ Downloading from URL...")
        print(f"   Target file: {video_filename}")

        try:
            # Use subprocess for better control
            result = subprocess.run(
                ['wget', '-O', video_filename, video_url, '--no-check-certificate', '-q', '--show-progress'],
                capture_output=True,
                text=True,
                timeout=300
            )

            if result.returncode == 0 and os.path.exists(video_filename):
                if os.path.getsize(video_filename) > 0:
                    video_path = video_filename
                    print(f"✅ Downloaded successfully: {video_filename}")
                else:
                    print(f"❌ Download failed: File is empty")
                    if os.path.exists(video_filename):
                        os.remove(video_filename)
            else:
                print(f"❌ Download failed: wget returned code {result.returncode}")
                # Try alternative method with curl
                print("\n🔄 Trying alternative download method (curl)...")
                result2 = subprocess.run(
                    ['curl', '-L', '-o', video_filename, video_url, '--silent', '--show-error'],
                    capture_output=True,
                    text=True,
                    timeout=300
                )

                if result2.returncode == 0 and os.path.exists(video_filename) and os.path.getsize(video_filename) > 0:
                    video_path = video_filename
                    print(f"✅ Downloaded successfully with curl: {video_filename}")
                else:
                    print(f"❌ Alternative download also failed")
                    print("   Please check if the URL is accessible and try again")

        except subprocess.TimeoutExpired:
            print("❌ Download timed out (>5 minutes). File may be too large.")
        except Exception as e:
            print(f"❌ Download error: {e}")

elif choice == "3":
    youtube_url = input("\nEnter YouTube URL (video or shorts): ").strip()

    if not youtube_url:
        print("❌ No URL provided")
    else:
        print("⬇️ Downloading from YouTube...")
        print("   Installing yt-dlp (if needed)...")

        # Install yt-dlp if not present
        subprocess.run(['pip', 'install', '-q', 'yt-dlp'], check=False)

        video_filename = "youtube_video.mp4"

        try:
            print(f"   Fetching video info...")

            # Download with yt-dlp
            result = subprocess.run(
                [
                    'yt-dlp',
                    '-f', 'best[ext=mp4]/best',  # Best quality MP4
                    '-o', video_filename,
                    '--no-playlist',
                    '--quiet',
                    '--progress',
                    youtube_url
                ],
                capture_output=True,
                text=True,
                timeout=600  # 10 minute timeout for YouTube
            )

            if result.returncode == 0 and os.path.exists(video_filename):
                if os.path.getsize(video_filename) > 0:
                    video_path = video_filename
                    print(f"✅ Downloaded successfully: {video_filename}")
                else:
                    print(f"❌ Download failed: File is empty")
                    if os.path.exists(video_filename):
                        os.remove(video_filename)
            else:
                print(f"❌ YouTube download failed")
                if result.stderr:
                    print(f"   Error: {result.stderr[:300]}")
                print("\n💡 Troubleshooting tips:")
                print("   - Make sure the video is public and not age-restricted")
                print("   - Try using Option 1 to upload the video manually")
                print("   - Check if the URL is correct")

        except subprocess.TimeoutExpired:
            print("❌ Download timed out (>10 minutes).")
        except Exception as e:
            print(f"❌ Download error: {e}")

else:
    print("⚠️ Invalid choice. Please choose option 1, 2, or 3.")

# Validate the video file
if video_path:
    if os.path.exists(video_path):
        file_size = os.path.getsize(video_path) / (1024*1024)  # MB
        print(f"\n📹 Video ready: {video_path} ({file_size:.1f} MB)")

        # Verify it's a valid video file
        import cv2
        cap = cv2.VideoCapture(video_path)
        if cap.isOpened():
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration = frame_count / fps if fps > 0 else 0
            print(f"   Duration: {duration:.1f} seconds")
            print(f"   FPS: {fps:.1f}")
            print(f"   Total frames: {frame_count:,}")
            cap.release()
        else:
            print("\n⚠️ Warning: Unable to read video file. It may be corrupted.")
            print("   Please try a different video or URL.")
            video_path = None
    else:
        print(f"\n❌ Error: File not found at {video_path}")
        video_path = None

if not video_path:
    print("\n❌ No valid video file available. Please run this cell again.")

📤 Choose how to get your video:

1. Upload from computer (recommended for small files < 100MB)
2. Download from URL (direct video file)
3. Download from YouTube URL

Enter choice (1/2/3): 3

Enter YouTube URL (video or shorts): https://www.youtube.com/shorts/QhlroYnundk
⬇️ Downloading from YouTube...
   Installing yt-dlp (if needed)...
   Fetching video info...
✅ Downloaded successfully: youtube_video.mp4

📹 Video ready: youtube_video.mp4 (2.2 MB)
   Duration: 7.6 seconds
   FPS: 30.0
   Total frames: 228


In [ ]:
# Using InstructBLIP for object-focused captioning
print("📝 Using InstructBLIP for captioning")
print("   Focuses on object attributes (colors, sizes, etc.)")
print("   Better than standard BLIP for object descriptions like 'black backpack', 'red shirt'")
use_object_detection = False  # Object detection mode disabled

Choose your captioning method:

1. Standard BLIP (faster, general scene captions)
2. Object Detection + BLIP (slower, object-focused)

Enter choice (1/2, default=1): 2

Using Object Detection + BLIP pipeline
   Detects objects: bags, laptops, helmets, phones, etc.


## Step 5: Process the Video

This will:
1. Extract frames from the video (removing redundant frames)
2. Generate captions using **InstructBLIP** (object-focused with colors and attributes)
3. Create embeddings for semantic search
4. Upload to Pinecone database

**Expected time:**
- 1 minute video: ~2-3 minutes with GPU
- 5 minute video: ~8-10 minutes with GPU
- CPU mode: 3-5x slower

**Caption Quality:**
- InstructBLIP provides detailed object descriptions with colors and attributes
- Example: "a person wearing a red shirt and blue jeans with a black backpack"

In [ ]:
import time
from datetime import datetime

if 'video_path' not in locals() or not video_path:
    print("❌ Please upload a video first (run the previous cell)")
else:
    # Set video name
    video_name = input("Enter a name for this video (or press Enter for auto-name): ").strip()
    if not video_name:
        video_name = f"video_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    print(f"\n🎬 Processing video: {video_name}")
    print("⏳ This will take a few minutes... Please wait.\n")
    print("=" * 60)

    start_time = time.time()

    try:
        # Process the video with InstructBLIP
        stats = engine.process_video(
            video_path=video_path,
            video_name=video_name,
            save_frames=False,  # Set to True to save frames
            upload_to_pinecone=True,
            use_object_detection=False  # Using InstructBLIP only (no Grounding DINO)
        )

        processing_time = time.time() - start_time

        print("\n" + "=" * 60)
        print("\n✅ VIDEO PROCESSING COMPLETE!\n")
        print(f"📊 Processing Statistics:")
        print(f"   Video name: {video_name}")
        print(f"   Frames extracted: {stats['total_frames_extracted']:,}")
        print(f"   Frames with captions: {stats['frames_with_captions']:,}")
        print(f"   Captions before dedupe: {stats.get('captions_before_dedupe', stats['frames_with_captions']):,}")
        print(f"   Unique embeddings: {stats.get('embeddings_generated', 0):,}")
        print(f"   ✅ Actually uploaded: {stats['embeddings_uploaded']:,}")
        print(f"   Processing time: {processing_time/60:.1f} minutes")
        print(f"\n   Frame reduction: {stats.get('frame_reduction_percent', 0):.1f}%")

        # Save video_name for next steps
        processed_video_name = video_name

    except Exception as e:
        print(f"\n❌ Error processing video: {e}")
        print("\nTroubleshooting tips:")
        print("- If GPU memory error: Restart runtime and try again")
        print("- If video format error: Convert video to MP4 format")

Enter a name for this video (or press Enter for auto-name): dino

🎬 Processing video: dino
⏳ This will take a few minutes... Please wait.



Processing frames:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/transformers/models/grounding_dino/processing_grounding_dino.py:93: FutureWarning: The key `labels` is will return integer ids in `GroundingDinoProcessor.post_process_grounded_object_detection` output since v4.51.0. Use `text_labels` instead to retrieve string object names.
  warnings.warn(self.message, FutureWarning)
ERROR:object_caption_pipeline:Error captioning object backpack: 'ObjectCaptionPipeline' object has no attribute '_score_attribute_caption'
ERROR:object_caption_pipeline:Error captioning object person student: 'ObjectCaptionPipeline' object has no attribute '_score_attribute_caption'
ERROR:object_caption_pipeline:Error captioning object person student: 'ObjectCaptionPipeline' object has no attribute '_score_attribute_caption'
ERROR:object_caption_pipeline:Error captioning object car: 'ObjectCaptionPipeline' object has no attribute '_score_attribute_caption'
ERROR:object_caption_

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading to Pinecone: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]




✅ VIDEO PROCESSING COMPLETE!

📊 Processing Statistics:
   Video name: dino
   Frames extracted: 5
   Frames with captions: 2
   Captions before dedupe: 2
   Unique embeddings: 1
   ✅ Actually uploaded: 1
   Processing time: 0.4 minutes

   Frame reduction: 60.0%


## Step 6: Search Your Video!

Search using natural language queries. The system uses **temporal bootstrapping** to find related objects automatically!

**Features in action:**
- 🔄 **Temporal Bootstrapping**: Finds related objects at similar timestamps
- 📊 **Adaptive Window**: Adjusts search window based on motion intensity
- ⚡ **Confidence-Aware**: Weights results by detection confidence

**Example queries:**
- "person walking" → Also finds "bag", "backpack" nearby
- "red shirt" → Also finds "blue bag", "person" at similar times
- "black bag" → Also finds related objects in the same scenes

In [ ]:
# Search with temporal bootstrapping (uses all 3 novel features!)
query = input("🔍 Enter your search query: ").strip()

print(f"\n🔍 Searching for: '{query}' with temporal bootstrapping...")
print("=" * 60)
print("✨ Using: Temporal Bootstrapping + Adaptive Window + Confidence-Aware Boosting\n")

# Use search_with_bootstrapping for intelligent search
results_dict = engine.search_with_bootstrapping(
    primary_query=query,
    auto_extract_related=True,  # Automatically find related objects
    top_k=5,
    similarity_threshold=0.5,
    video_filter=processed_video_name if 'processed_video_name' in locals() else None
)

# Display results
if results_dict and query in results_dict:
    primary_results = results_dict[query]
    related_queries = [q for q in results_dict.keys() if q != query]
    
    print(f"✅ Primary query '{query}' found {len(primary_results)} results:\n")
    for i, result in enumerate(primary_results, 1):
        print(f"{i}. ⏱️ Timestamp: {result['time_formatted']}")
        print(f"   📝 Caption: {result['caption']}")
        print(f"   📊 Confidence: {result['similarity_score']:.1%}")
        print(f"   🎥 Video: {result['video_name']}")
        print()
    
    # Show boosted related results
    if related_queries:
        print(f"\n🔄 Temporal Bootstrapping also found related objects:\n")
        for related_q in related_queries[:3]:  # Show top 3 related queries
            related_results = results_dict.get(related_q, [])
            if related_results:
                print(f"   📌 Related: '{related_q}' ({len(related_results)} boosted results)")
                for result in related_results[:2]:  # Show top 2
                    boost_info = f" (boosted from {result.get('original_score', 0):.1%})" if 'original_score' in result else ""
                    print(f"      └─ {result['time_formatted']}: {result['caption'][:50]}... ({result['similarity_score']:.1%}{boost_info})")
                print()
else:
    print("\n❌ No results found. Try:")
    print("   - Different search terms")
    print("   - More general queries")
    print("   - Lowering the similarity threshold")

## Step 7: Batch Search (Multiple Queries)

Search for multiple things at once! Each query uses temporal bootstrapping automatically.

In [ ]:
# Define multiple queries
queries = [
    "person walking",
    "black bag",
    "red shirt",
    "outdoor scene"
]

print("🔍 Running batch search with temporal bootstrapping...\n")
print("=" * 60)
print("✨ Each query uses: Temporal Bootstrapping + Adaptive Windows + Confidence-Aware\n")

batch_results = {}
for query in queries:
    # Use temporal bootstrapping for each query
    results_dict = engine.search_with_bootstrapping(
        primary_query=query,
        auto_extract_related=True,
        top_k=3
    )
    
    if results_dict and query in results_dict:
        batch_results[query] = results_dict[query]

# Display results
for query, results in batch_results.items():
    print(f"\n📌 Query: '{query}'")
    print(f"   Found {len(results)} results")

    if results:
        for result in results[:2]:  # Show top 2
            print(f"   └─ {result['time_formatted']} - {result['caption'][:50]}... ({result['similarity_score']:.0%})")
    else:
        print("   └─ No results")

print("\n" + "=" * 60)
print("\n✨ Note: Temporal bootstrapping automatically found related objects for each query!")

## Step 8: Advanced Search (Filter by Video)

Search with video filter - still uses temporal bootstrapping for intelligent results!

In [ ]:
# Advanced search with temporal bootstrapping (filter by video)
query = input("🔍 Enter search query: ").strip()

# Optional: Filter by video name
video_filter = None
if 'processed_video_name' in locals():
    filter_video = input(f"\nSearch only in '{processed_video_name}'? (y/n): ").lower() == 'y'
    if filter_video:
        video_filter = processed_video_name

# Perform search with temporal bootstrapping
print(f"\n🔍 Searching with temporal bootstrapping...")
print("✨ Features: Adaptive windows + Confidence-aware boosting\n")

results_dict = engine.search_with_bootstrapping(
    primary_query=query,
    auto_extract_related=True,
    top_k=10,
    similarity_threshold=0.4,  # Lower threshold for more results
    video_filter=video_filter
)

# Display results
if results_dict and query in results_dict:
    primary_results = results_dict[query]
    print(f"\n✅ Found {len(primary_results)} results for '{query}':\n")
    for i, result in enumerate(primary_results, 1):
        print(f"{i}. {result['time_formatted']} - {result['caption'][:60]}... ({result['similarity_score']:.1%})")
    
    # Show related objects found
    related_queries = [q for q in results_dict.keys() if q != query]
    if related_queries:
        print(f"\n🔄 Also found {len(related_queries)} related object types at similar timestamps!")
else:
    print("\n❌ No results found.")

## Step 9: Interactive Search Interface


In [ ]:
print("🎯 INTERACTIVE VIDEO SEARCH (with Temporal Bootstrapping)")
print("=" * 60)
print("✨ Features active: Temporal Bootstrapping + Adaptive Windows + Confidence-Aware")
print("Enter your search queries (type 'quit' to exit)\n")

import cv2
from IPython.display import display
from PIL import Image
import numpy as np

def display_frame_image(video_name: str, timestamp_sec: float):
    """Extract and display a frame image from the video at the given timestamp."""
    # Try to resolve the exact video path from the engine (preferred), else fall back to last uploaded path
    vpath = None
    try:
        if hasattr(engine, 'video_paths') and video_name in engine.video_paths:
            vpath = engine.video_paths.get(video_name)
    except Exception:
        vpath = None
    if not vpath and 'video_path' in globals():
        vpath = video_path
    if not vpath:
        print("   ⚠️ Unable to resolve video path to display frame.")
        return
    
    cap = cv2.VideoCapture(vpath)
    if not cap.isOpened():
        print("   ⚠️ Unable to open video to extract frame.")
        return
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    frame_num = int(max(0, timestamp_sec) * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    ok, frame = cap.read()
    cap.release()
    if not ok or frame is None:
        print("   ⚠️ Could not read frame at timestamp.")
        return
    # Convert BGR -> RGB and display
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(rgb)
    display(img)

while True:
    query = input("\n🔍 Search: ").strip()

    if query.lower() in ['quit', 'exit', 'q']:
        print("\n👋 Goodbye!")
        break

    if not query:
        continue

    # Use temporal bootstrapping search
    results_dict = engine.search_with_bootstrapping(
        primary_query=query,
        auto_extract_related=True,
        top_k=5
    )

    if results_dict and query in results_dict:
        results = results_dict[query]
        related_queries = [q for q in results_dict.keys() if q != query]
        
        print(f"\n✅ Found {len(results)} results for '{query}':")
        for i, result in enumerate(results, 1):
            score_emoji = "🟢" if result['similarity_score'] > 0.7 else "🟡" if result['similarity_score'] > 0.5 else "🟠"
            print(f"\n{i}. {score_emoji} {result['time_formatted']} ({result['similarity_score']:.0%})")
            print(f"   {result['caption']}")
            # Show the image for the first (top-1) result to avoid heavy output
            if i == 1:
                print("   🖼️ Frame preview:")
                try:
                    display_frame_image(result.get('video_name', ''), float(result.get('timestamp', 0.0)))
                except Exception as e:
                    print(f"   ⚠️ Preview error: {e}")
        
        if related_queries:
            print(f"\n   🔄 Also found: {', '.join(related_queries[:3])} at similar times")
    else:
        print("\n❌ No results found. Try a different query.")